<a href="https://colab.research.google.com/github/Aashay7/BERT/blob/master/DocProductPresentationV6_0_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#@title Install Faiss, TF 2.0, and our Github. Double Click to see code

#To use CPU FAISS use
!wget  https://anaconda.org/pytorch/faiss-cpu/1.2.1/download/linux-64/faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
#To use GPU FAISS use
# !wget  https://anaconda.org/pytorch/faiss-gpu/1.2.1/download/linux-64/faiss-gpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
!tar xvjf faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2 # pending---
!cp -r lib/python3.6/site-packages/* /usr/local/lib/python3.6/dist-packages/
!pip install mkl

!pip install tensorflow-gpu==2.0.0-alpha0
import tensorflow as tf
!pip install https://github.com/re-search/DocProduct/archive/v0.2.0_dev.zip
!pip install gpt2-estimator
!pip install pyarrow


--2019-06-30 07:49:34--  https://anaconda.org/pytorch/faiss-cpu/1.2.1/download/linux-64/faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
Resolving anaconda.org (anaconda.org)... 104.17.92.24, 104.17.93.24, 2606:4700::6811:5c18, ...
Connecting to anaconda.org (anaconda.org)|104.17.92.24|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://binstar-cio-packages-prod.s3.amazonaws.com/5a15c9c5c376961204909d87/5aa7f0a65571b411e5c259be?response-content-disposition=attachment%3B%20filename%3D%22faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2%22%3B%20filename%2A%3DUTF-8%27%27faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2&response-content-type=application%2Fx-tar&X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Expires=60&X-Amz-Date=20190630T074934Z&X-Amz-SignedHeaders=host&X-Amz-Security-Token=AgoJb3JpZ2luX2VjED0aCXVzLWVhc3QtMSJIMEYCIQDJpg2xUo5c1dszQj2n7qeRdvdKJL6LYBYIv86KnVTmCgIhAOW152TDfo4IngXN5zScRdaJaR6HrRpRs8u9MIt7i4NSKuMDCIb%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEQABoMNDU1ODY0MDk4Mzc4Igy

In [0]:
!ls

faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2    info
faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2.1  lib
faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2.2  sample_data


In [0]:
#@title Downaload all model checkpoints, and question/answer data. Double click to see this code

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                
import os
import requests

import urllib.request

# Download the file from `url` and save it locally under `file_name`:
urllib.request.urlretrieve('https://github.com/naver/biobert-pretrained/releases/download/v1.0-pubmed-pmc/biobert_v1.0_pubmed_pmc.tar.gz', 'BioBert.tar.gz')

if not os.path.exists('BioBertFolder'):
    os.makedirs('BioBertFolder')
    
import tarfile
tar = tarfile.open("BioBert.tar.gz")
tar.extractall(path='BioBertFolder/')
tar.close()

file_id = '1uCXv6mQkFfpw5txGnVCsl93Db7t5Z2mp'

download_file_from_google_drive(file_id, 'Float16EmbeddingsExpanded5-27-19.pkl')

file_id = 'https://onedrive.live.com/download?cid=9DEDF3C1E2D7E77F&resid=9DEDF3C1E2D7E77F%2132792&authkey=AEQ8GtkcDbe3K98'
    
urllib.request.urlretrieve( file_id, 'DataAndCheckpoint.zip')

if not os.path.exists('newFolder'):
    os.makedirs('newFolder')

import zipfile
zip_ref = zipfile.ZipFile('DataAndCheckpoint.zip', 'r')
zip_ref.extractall('newFolder')
zip_ref.close()

In [0]:
!ls

BioBertFolder	       faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2  lib
BioBert.tar.gz	       Float16EmbeddingsExpanded5-27-19.pkl	   newFolder
DataAndCheckpoint.zip  info					   sample_data


In [0]:
#@title Load model weights and Q&A data. Double click to see code

from docproduct.predictor import RetreiveQADoc

pretrained_path = 'BioBertFolder/biobert_v1.0_pubmed_pmc/'
# ffn_weight_file = None
bert_ffn_weight_file = 'newFolder/models/bertffn_crossentropy/bertffn'
embedding_file = 'Float16EmbeddingsExpanded5-27-19.pkl'

doc = RetreiveQADoc(pretrained_path=pretrained_path,
ffn_weight_file=None,
bert_ffn_weight_file=bert_ffn_weight_file,
embedding_file=embedding_file)

Failed to load GPU Faiss: No module named 'faiss.swigfaiss_gpu'
Faiss falling back to CPU-only.


In [0]:
#@title Type in your question (512 word limit) and search search parameters

question_text = "I have multiple skin rashes in summer"  #@param {type:"string"}

search_similarity_by = 'answer'  #@param ['answer', "question"]

number_results_toReturn=10 #@param {type:"number"}

answer_only=True #@param ["False", "True"] {type:"raw"}

returned_results = doc.predict( question_text ,
                  search_by=search_similarity_by, topk=number_results_toReturn, answer_only=answer_only)

print('')
for jk in range(len(returned_results)):
    print("Result ", jk+1)
    print(returned_results[jk])
    print('')


1it [00:00,  2.95it/s]


Result  1
there are many types and causes of skin rashes. several types of inflammatory conditions of the skin allergic reactions and sometimes bacterial viral or fungal infections can cause rashes on the skin. insect bites can also be the cause of a rash on the skin. have it evaluated by a professional and get the appropriate treatment.

Result  2
with your history of multible medical conditions and you have to on few medications and the rash you described it is very difficult to say what is causing the bumps on your arms and thighs i will suggest you consult a dematologist to find what these bumps are due to

Result  3
hi. these rashes seem to be dermatitis and skin infection. apply local creams and take anti fungal tablets. try using a different soap. get blood checked if its spreading over the body. steroid creams can be applied but only after a dermatologist opinion. thanks

Result  4
hi dear. you are right even i dont feel this is a allergic ras or an tss to me it seems to be a 